In [18]:
# https://www.edureka.co/blog/web-scraping-with-python/
# https://stackoverflow.com/questions/34519746/using-python-docx-to-update-cell-content-of-a-table
# https://www.lambdatest.com/blog/edge-driver/

from docx import Document, enum
from docx.shared import Pt
import requests


doc_name = "Vorlage_S.25.docx"
rows_available = 14 # both sections need to have the same number of rows available
start_row_club_goalgetter_1 = 1
start_row_club_goalgetter_2 = rows_available + start_row_club_goalgetter_1 + 1

# URLS needs to be updated every season
club_goalscorers_config = [
{"url": "https://api.fupa.net/v1/teams/sg-mallersdorf-grafentr-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_1, "start_column": 0 },              
{"url": "https://api.fupa.net/v1/teams/tsv-pilsting-m1-2023-24/player-stats",  "start_row": start_row_club_goalgetter_2, "start_column": 0 },              
{"url": "https://api.fupa.net/v1/teams/sv-wendelskirchen-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_1, "start_column": 2 },
{"url": "https://api.fupa.net/v1/teams/fc-wallersdorf-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_2, "start_column": 2 },             
{"url": "https://api.fupa.net/v1/teams/ssv-1983-weng-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_1, "start_column": 4 },
{"url": "https://api.fupa.net/v1/teams/sv-wallkofen-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_2, "start_column": 4 },
{"url": "https://api.fupa.net/v1/teams/spvgg-haberskirchen-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_1, "start_column": 6 },        
{"url": "https://api.fupa.net/v1/teams/fc-ottering-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_2, "start_column": 6 },       
{"url": "https://api.fupa.net/v1/teams/sv-grosskoellnbach-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_1, "start_column": 8 },   
{"url": "https://api.fupa.net/v1/teams/tsv-mamming-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_2, "start_column": 8 },  
{"url": "https://api.fupa.net/v1/teams/tsv-hofkirchen-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_1, "start_column": 10 }, 
{"url": "https://api.fupa.net/v1/teams/sg-pfaffenberg-oberlindhart-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_2, "start_column": 10 }, 
{"url": "https://api.fupa.net/v1/teams/fc-gottfrieding-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_1, "start_column": 12 },
{"url": "https://api.fupa.net/v1/teams/sg-griesbach-steinberg-m1-2023-24/player-stats", "start_row": start_row_club_goalgetter_2, "start_column": 12 } ]

def print_data():
    # Fetch and sort players based on goals, assists, and number of games
    goalscorers = fetch_and_sort_goalscorers()

    # Print the sorted list of players based on goals, assists, and number of games
    if goalscorers:
        print("Sorted list of players based on goals, assists, and number of games:")
        print("{:<8} {:<20} {:<30} {:<6} {:<8} {:<8}".format('Position', 'Name', 'Club', 'Goals', 'Assists', 'Matches'))
        for player in goalscorers:
            print("{:<8} {:<20} {:<30} {:<6} {:<8} {:<8}".format(player['position'], player['name'], player['club'], player['goals'], player['assists'], player['matches']))


    # Fetch and sort players based on goals
    sorted_players = fetch_and_sort_players("https://api.fupa.net/v1/teams/sg-mallersdorf-grafentr-m1-2023-24/player-stats")

    # Print the sorted list of players based on goals
    if sorted_players:
        print("Sorted list of club players based on goals:")
        for player, goals in sorted_players:
            print(f"{player}: {goals}")

# fix name mapping - data based on goalgetter_url
def fix_club_name(name):
    if name == "SG Pfaffenberg":
        name = "SG Pfaffenberg-Oberlindhart"
    if name == "Großköllnbac":
        name = "SV Großköllnbach"
    if name == "Hofkirchen":
        name = "TSV Hofkirchen"
    if name == "SG Mallersdorf":
        name = "SG Mallersdorf-Grafentr."
    if name == "Wallersdorf":
        name = "FC Wallersdorf"
    if name == "Weng":
        name = "SSV Weng"
    if name == "Haberskirch.":
        name = "SpVgg Haberskirchen"
    if name == "Mamming":
        name = "TSV Mamming"
    if name == "Wallkofen":
        name = "SV Wallkofen"
    if name == "Ottering":
        name = "FC Ottering"
    if name == "Gottfrieding":
        name = "FC Gottfrieding"
    if name == "SG Griesbach":
        name = "SG Griesbach-Steinberg"
    if name == "Pilsting":
        name = "TSV Pilsting"
    if name == "Wendelskir.":
        name = "SV Wendelskirchen"
    return name

def set_font(doc, font_name):
    style = doc.styles['Normal']
    font = style.font
    font.name = font_name

def update_goalgetter(doc):

    goalscorers = fetch_and_sort_goalscorers()
    
    table = doc.tables[1]

    # iterate word table
    for x in range(19):
        # skip empty rows
        if x % 2 == 0:
            for y in range(7):
                position_in_word_table = int(x/2)
                if y == 1 :
                    table.cell(x,y).text = goalscorers[position_in_word_table]['name']
                elif y == 2:
                    table.cell(x,y).text = goalscorers[position_in_word_table]['club']
                elif y == 3:
                    table.cell(x,y).text = goalscorers[position_in_word_table]['goals']
                elif y == 5:
                    table.cell(x,y).text = goalscorers[position_in_word_table]['assists']
                table.cell(x,y).paragraphs[0].runs[0].font.size = Pt(8)
                table.cell(x,y).paragraphs[0].runs[0].font.bold = True

def update_club_goalgetter(doc):
    table = doc.tables[0]

    for i in range(len(club_goalscorers_config)):

        goalscorers = fetch_and_sort_club_goalscorers(club_goalscorers_config[i]["url"])
        print(goalscorers)
        for j in range(len(goalscorers)):
            cell_name = table.cell(club_goalscorers_config[i]["start_row"] + j, club_goalscorers_config[i]["start_column"])
            cell_name.text = goalscorers[j]["name"]
            cell_name.paragraphs[0].runs[0].font.size = Pt(6)
            cell_goals = table.cell(club_goalscorers_config[i]["start_row"] + j, club_goalscorers_config[i]["start_column"] + 1)
            cell_goals.text = goalscorers[j]["goals"]
            cell_goals.paragraphs[0].runs[0].font.size = Pt(6)
            cell_goals.paragraphs[0].alignment = enum.text.WD_ALIGN_PARAGRAPH.CENTER
            # print("Name: ", goalscorers[j]["name"], " with goals: ", goalscorers[j]["goals"])

# Function to fetch data from the API and return a sorted list of players based on goals, assists, and number of games
def fetch_and_sort_goalscorers():
    # Fetch data from the API
    response = requests.get("https://api.fupa.net/v1/competitions/kreisklasse-dingolfing/seasons/current/scorers?limit=10")
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse JSON response
        player_stats = response.json()
        
        # Sort the list of players based on goals, assists, and number of games
        sorted_players = sorted(player_stats, key=lambda x: (x['statistics']['goals'], x['statistics']['assists'], x['statistics']['matches']), reverse=True)
        
        # Create a list of dictionaries containing player details
        sorted_players_list = []
        for index, player in enumerate(sorted_players):
            player_info = {
                'position': index + 1,
                'name': player['player']['firstName'][0] + '. ' + player['player']['lastName'], # combine first letter of first name with lastname
                'club': fix_club_name(player['team']['name']['full']), # adapt club name, so it matches the document format
                'goals': str(player['statistics']['goals']), # convert number to string
                'assists': str(player['statistics']['assists']), # convert number to string
                'matches': player['statistics']['matches']
            }
            sorted_players_list.append(player_info)
        
        return sorted_players_list
    else:
        # If the request was not successful, print an error message
        print("Error fetching data from API")
        return None

# Function to fetch data from the API and return a sorted list of players based on goals
def fetch_and_sort_club_goalscorers(api_url):
    # Fetch data from the API
    response = requests.get(api_url)
    
    # Check if the request was successful
    if response.status_code == 200:
        # Parse JSON response
        player_stats = response.json()
        
        # Sort the list of players based on goals
        sorted_players = sorted(player_stats, key=lambda x: x['goals'], reverse=True)
        
        # Create a list containing player name and goals
        player_goals_list = [{'name': player['firstName'][0] + '. ' + player['lastName'], 'goals': str(player['goals'])} for player in sorted_players if player['goals'] > 0]
        
        return player_goals_list
    else:
        # If the request was not successful, print an error message
        print("Error fetching data from API")
        return None

def run():
    doc = Document(doc_name)
    set_font(doc, 'Arial')
    update_club_goalgetter(doc)
    update_goalgetter(doc)
    doc.save("_S.25.docx")

run()


[{'name': 'F. Prebeck-Sanchez', 'goals': '16'}, {'name': 'M. Löffler', 'goals': '14'}, {'name': 'E. Daffner', 'goals': '8'}, {'name': 'K. Prebeck-Sanchez', 'goals': '7'}, {'name': 'P. Kleinevoss', 'goals': '6'}, {'name': 'L. Löffler', 'goals': '4'}, {'name': 'L. Gruber', 'goals': '3'}, {'name': 'C. Grauschopf', 'goals': '2'}, {'name': 'S. Draszczyk', 'goals': '1'}, {'name': 'B. Schütz', 'goals': '1'}]
Name:  F. Prebeck-Sanchez  with goals:  16
Name:  M. Löffler  with goals:  14
Name:  E. Daffner  with goals:  8
Name:  K. Prebeck-Sanchez  with goals:  7
Name:  P. Kleinevoss  with goals:  6
Name:  L. Löffler  with goals:  4
Name:  L. Gruber  with goals:  3
Name:  C. Grauschopf  with goals:  2
Name:  S. Draszczyk  with goals:  1
Name:  B. Schütz  with goals:  1
[{'name': 'K. Gegenfurtner', 'goals': '10'}, {'name': 'J. Aulinger', 'goals': '6'}, {'name': 'T. Hofmann', 'goals': '4'}, {'name': 'R. Ritzinger', 'goals': '2'}, {'name': 'M. Käser', 'goals': '2'}, {'name': 'R. Siegerstetter', 'goa